In [0]:
import numpy as np
import tensorflow  as tf
from tensorflow.contrib.layers import fully_connected
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
import librosa
from google.colab import files
from google.colab import drive
import glob
import pickle
from statistics import mean
import keras
from keras.layers.core import Dense, Permute, Reshape
from keras.layers.convolutional import Convolution2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.regularizers import l2
from sklearn.metrics import confusion_matrix


np.random.seed(42)

Using TensorFlow backend.


In [0]:
drive.mount('/content/Drive/',force_remount =  True)

Mounted at /content/Drive/


In [0]:
dirpath = "/content/Drive/My Drive/DL project/New Files/Saturyay Data/" ####Change

In [0]:
#Loading training pickle files
sound_lists_pkl_in = open(dirpath+"Catalan_stft_2500.pkl", "rb")
ca_data_stft = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"French_stft_2500.pkl", "rb")
fr_data_stft = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"English_stft_2500.pkl", "rb")
en_data_stft = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"German_stft_2500.pkl", "rb")
de_data_stft = pickle.load(sound_lists_pkl_in)

sound_lists_pkl_in = open(dirpath+"Italian_stft_2500.pkl", "rb")
it_data_stft = pickle.load(sound_lists_pkl_in)



In [0]:
# def no_of_row(dict):
#   rows = []
#   for i in range(len(dict['stft'])):
#     r = dict['stft'][i].shape[0]
#     rows.append(r)
#   print(min(rows),max(rows))
  

In [0]:
# no_of_row(it_data_stft)
# no_of_row(de_data_stft)
# no_of_row(en_data_stft)
# no_of_row(fr_data_stft)
# no_of_row(ca_data_stft)

In [0]:
def data_creation(dict):
  data=[]
  stft=[]
  for i in range(2500):
    s = dict['stft'][i]
    stft.append(s)
  stft = np.array(stft)
  for i in range(len(stft)):
    ele = np.array([stft[i],dict['labels'][i]])
    data.append(ele)
  tr_data = np.array(data[:2000])
  te_data = np.array(data[2000:])
  return tr_data, te_data
    
                   
  

In [0]:
ca_data_stft_tr, ca_data_stft_te = data_creation(ca_data_stft)

In [0]:
len(ca_data_stft_te)

500

In [0]:
#Train Data
# ca_data_stft_tr, ca_data_stft_te = data_creation(ca_data_stft)
en_data_stft_tr, en_data_stft_te = data_creation(en_data_stft)
fr_data_stft_tr, fr_data_stft_te = data_creation(fr_data_stft)
it_data_stft_tr, it_data_stft_te = data_creation(it_data_stft)
de_data_stft_tr, de_data_stft_te = data_creation(de_data_stft)

In [0]:
de_data_stft_tr[0]

array([array([[7.2276924e-04, 4.1775490e-04, 1.1588744e-03, ..., 3.0463249e-05,
        3.5601832e-05, 3.5377976e-05],
       [4.0209602e-04, 9.5266296e-04, 9.0733066e-04, ..., 3.0605181e-06,
        6.5928771e-07, 2.4030962e-07],
       [2.4213810e-03, 1.5910225e-03, 2.2536061e-04, ..., 4.0215841e-06,
        4.7475086e-07, 2.8739476e-07],
       ...,
       [1.3734745e-02, 3.9453715e-02, 3.7081767e-02, ..., 4.1201775e-04,
        4.1084143e-04, 5.6367051e-05],
       [1.8951178e-03, 2.7881728e-03, 1.5420970e-02, ..., 4.8314180e-04,
        2.8682681e-04, 1.2787881e-04],
       [2.5847754e-02, 4.2112991e-03, 3.8081575e-02, ..., 1.3598987e-04,
        1.2379725e-04, 4.6535810e-05]], dtype=float32),
       'German'], dtype=object)

In [0]:
final_data_tr = np.concatenate((ca_data_stft_tr, fr_data_stft_tr, en_data_stft_tr, de_data_stft_tr, it_data_stft_tr), axis = 0)

In [0]:
final_data_te = np.concatenate((ca_data_stft_te, fr_data_stft_te, en_data_stft_te, de_data_stft_te, it_data_stft_te), axis = 0)

In [0]:
#QC
print(np.unique(final_data_tr[:,1], return_counts = True))
print(np.unique(final_data_te[:,1], return_counts = True))


(array(['Catalan', 'English', 'French', 'German', 'Italian'], dtype=object), array([2000, 2000, 2000, 2000, 2000]))
(array(['Catalan', 'English', 'French', 'German', 'Italian'], dtype=object), array([500, 500, 500, 500, 500]))


In [0]:
#QC
print(final_data_tr.shape)
print(final_data_te.shape)

(10000, 2)
(2500, 2)


In [0]:
del(ca_data_stft_te)
del(it_data_stft_te)
del(fr_data_stft_te)
del(en_data_stft_te)
del(de_data_stft_te)

In [0]:
del(ca_data_stft_tr)
del(it_data_stft_tr)
del(fr_data_stft_tr)
del(en_data_stft_tr)
del(de_data_stft_tr)

In [0]:
np.random.shuffle(final_data_tr)
np.random.shuffle(final_data_te)

In [0]:
#Labels

labels_tr =[]
for i in range(len(final_data_tr)):

      labels_tr.append(final_data_tr[i][1])  

labels_tr = np.array(labels_tr)
labels_tr

classnames, indices = np.unique(labels_tr, return_inverse=True)
print(indices)
print(classnames)

final_labels_tr = indices

[3 1 0 ... 1 0 3]
['Catalan' 'English' 'French' 'German' 'Italian']


In [0]:
labels_te =[]
for i in range(len(final_data_te)):

      labels_te.append(final_data_te[i][1])  

labels_te = np.array(labels_te)
labels_te

classnames, indices = np.unique(labels_te, return_inverse=True)
print(indices)
print(classnames)

final_labels_te = indices

[3 1 2 ... 2 3 3]
['Catalan' 'English' 'French' 'German' 'Italian']


In [0]:
final_X_tr =[]
for i in range(len(final_data_tr)):

      final_X_tr.append(final_data_tr[i][0])  

final_X_tr = np.array(final_X_tr)
final_X_tr[0].shape


(247, 129)

In [0]:
final_X_te =[]
for i in range(len(final_data_te)):

      final_X_te.append(final_data_te[i][0])  

final_X_te = np.array(final_X_te)
final_X_te[0].shape


(247, 129)

### Keras - CRNN

In [0]:
from keras.layers.core import Dense, Permute, Reshape, Flatten
from keras.layers import Dense, Permute, Reshape, Input
from keras.layers.convolutional import Convolution2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import LSTM
from keras.layers import Dropout, TimeDistributed
from keras import layers
from keras.models import Sequential
from keras.regularizers import l2
from keras.utils import to_categorical
from keras.layers import Input
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.layers import GlobalAveragePooling2D
from keras.applications.mobilenet import MobileNet
from keras.models import Model
import keras

In [0]:
# input_shape = (40,972,1)
input_shape2 = (247,129,1)


In [0]:
# model.summary()

In [0]:
del(final_data_tr)
del(final_data_te)

In [0]:
tx = np.array(final_X_tr[:8000])
cvx = np.array(final_X_tr[8000:])
ty = np.array(final_labels_tr[:8000])
cvy = np.array(final_labels_tr[8000:])

In [0]:
del(final_labels_tr)
del(final_X_tr)

In [0]:
cvy_cat = to_categorical(cvy)
ty_cat = to_categorical(ty)

In [0]:
def corr_shape(arr):
  new_arr = []
  for i in range(len(arr)):
    new_arr.append(arr[i])
  new_arr = np.array(new_arr)
  print(new_arr.shape)
  return new_arr

In [0]:
tx = corr_shape(tx)
cvx = corr_shape(cvx)

(8000, 247, 129)
(2000, 247, 129)


In [0]:
#Resnet Model
base_model = MobileNet(weights = None, include_top= False, input_shape=input_shape2, alpha=1.0, dropout = 0.9)

# for layer in base_model.layers:
#   layer.trainable = False
 
x = base_model.output
# x = GlobalAveragePooling2D()(x)

x = Dropout(0.9)(x)

# x = Permute((2, 1, 3))(x)

# bs, x, y, c = model.layers[-1].output_shape

# model.add(Reshape([x,y*c]))

# model.add(LSTM(512, activation="relu",dropout = 0.7, return_sequences=False))

# x = Permute((2, 1, 3))(x)

_x, _y, _c = [int(s) for s in x._shape[1:]]
x = Reshape((_x, _y*_c))(x)

# x = LSTM(256, return_sequences=False)(x)

# x = LSTM(512, activation="tanh", dropout = 0.7, return_sequences=False)(x)

x = Bidirectional(LSTM(512, return_sequences=False, dropout = 0.7), merge_mode="concat")(x)
# x = Flatten()(x)

# x = Dense(1024, activation="relu")(x)

predictions = Dense(5, activation="softmax")(x)

model = Model(inputs = base_model.input, output = predictions)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.adam(lr=0.0005),
             metrics=["accuracy"])

In [0]:
model.fit(tx.reshape(-1,247,129,1), ty_cat,
          batch_size=300,
          epochs=100,
          verbose=1,
          validation_data=(cvx.reshape(-1,247,129,1),cvy_cat)
         )
          

Instructions for updating:
Use tf.cast instead.
Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 51s 6ms/step - loss: 1.6866 - acc: 0.2189 - val_loss: 1.8726 - val_acc: 0.2520
Epoch 2/100
8000/8000 [==============================] - 36s 5ms/step - loss: 1.6580 - acc: 0.2279 - val_loss: 1.9182 - val_acc: 0.2470
Epoch 3/100
8000/8000 [==============================] - 35s 4ms/step - loss: 1.6432 - acc: 0.2468 - val_loss: 1.8763 - val_acc: 0.2465
Epoch 4/100
8000/8000 [==============================] - 36s 4ms/step - loss: 1.6305 - acc: 0.2466 - val_loss: 1.7874 - val_acc: 0.2445
Epoch 5/100
8000/8000 [==============================] - 36s 4ms/step - loss: 1.6112 - acc: 0.2555 - val_loss: 1.7547 - val_acc: 0.2590
Epoch 6/100
8000/8000 [==============================] - 36s 4ms/step - loss: 1.5957 - acc: 0.2764 - val_loss: 1.7676 - val_acc: 0.2570
Epoch 7/100
8000/8000 [==============================] - 36s 4ms/step - loss: 1.5874 - a

In [0]:
# model = load_model('Drive/My Drive/DL project/my_model.h5') ####

NameError: ignored

In [0]:
y_pred = model.predict(cvx.reshape(-1,247,129,1))
matrix = confusion_matrix(cvy_cat.argmax(axis=1), y_pred.argmax(axis=1))

In [0]:
  matrix

array([[137,  35,  16, 101, 111],
       [ 34, 126,  42, 112,  83],
       [ 15,  77, 113, 149,  29],
       [ 31,  43,  38, 229,  78],
       [ 21,  20,   1,  45, 314]])